## RSA密码破译project 实现代码
---
通过实验以下十余种攻击方法，共破解出除data5 9 13 14 17以外的16组数据，得到11组明文，串联如下：
My secret is a famous saying of Albert Einstein. That is"Logic will get\<TODO:message09>m A to B. Imagin\<TODO:message12-message??>

（尽管已经不属于RSA破译的范畴，但在得知以上信息的情况下，已经可以根据a famous saying of Albert Einstein搜索到完整的这句话并推测出剩余的明文）

In [8]:
from factordb.factordb import FactorDB
import libnum
import gmpy2
import primefac
def read_file(file_name):
    """
    读取data数据。
    :param file_name:
    :return: (N, e, c)列表
    """
    with open(file_name) as file:
        data = file.readline(1024)
        N = data[0:256]
        e = data[256:512]
        c = data[512:768]
        return map(lambda x: (int)(x, base=16), (N, e, c))

def read_multiple_file(file_ids):
    """
    批量读取data数据。
    :param file_ids: data的编号
    :return: (Ns, es, cs)列表
    """
    Ns, es, cs = [], [], []
    for i in file_ids:
        N, e, c = read_file("./dataset/data"+str(i))
        Ns.append(N)
        cs.append(c)
        es.append(e)
    return Ns, es, cs

### 共模攻击
可以解出data0和data4
解密结果：b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00My secre'
该部分明文：My secre

In [3]:
def CMAAttack(N, es, cs):
    """
    共模攻击。
    原理：两组数据使用了相同的N，令e1s1+e2s2=1，则m=c1^s1*c2^s2 mod N
    :param N:
    :param es:
    :param cs:
    :return:明文m（数字形式）
    """
    assert len(es) == 2 and len(cs) == 2
    u, v, gcd = libnum.xgcd(es[0], es[1])
    if gcd != 1:
        raise Exception("e1 is not prime to e2")
    m = pow(cs[0], u, N) * pow(cs[1], v, N) % N
    return m

Ns, es, cs = read_multiple_file([0, 4])
assert Ns[0] == Ns[1]
m = CMAAttack(Ns[0], es, cs)
print(libnum.n2s(m))

b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00My secre'


### 模不互素攻击
可以解出data1和data18
解密结果：
data1: b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x0b\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00. Imagin'
data18: b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00m A to B'

In [4]:
def MNPAttack(Ns, es, cs):
    """
    模不互素攻击。
    原理：两组数据使用的N不互素，则它们的最大公因数是它们各自的一个因子。
    :param Ns:
    :param es:
    :param cs:
    :return:明文列表ms（数字形式）
    """
    p = libnum.gcd(*Ns)
    assert p != 1
    qs = [N // p for N in Ns]
    ds = [libnum.invmod(e, (p - 1) * (q - 1)) for e, q in zip(es, qs)]
    ms = [pow(c, d, n) for c, d, n in zip(cs, ds, Ns)]
    return ms

Ns, es, cs = read_multiple_file([1, 18])
ms = MNPAttack(Ns, es, cs)
for m in ms:
    print(libnum.n2s(m))

b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x0b\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00. Imagin'
b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\n\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00m A to B'


### 查询数据库
可以解出data2,data6,data19
解密结果：
data2: b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 That is'
data6: b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 "Logic '
data19: b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00instein.'

In [5]:
def DatabaseAttack(N, e, c):
    """
    在factorDB数据库中查询N能否被直接分解。
    解出data2 data6 data19
    :param N:
    :param e:
    :param c:
    :return: 明文m（数字形式）
    """
    factor = FactorDB(N)
    factor.connect()
    res = factor.get_factor_list()
    print(res)
    if (res is None) or (len(res) > 2) or factor.get_status() == 'C':
        raise Exception(f"DatabaseAttack fails for N {N}")
    p, q = res[0], res[1]
    r = (p - 1) * (q - 1)
    d = libnum.invmod(e, r)
    m = pow(c, d, N)
    return m

Ns, es, cs = read_multiple_file([2, 6, 19])
for N, e, c in zip(Ns, es, cs):
    m = DatabaseAttack(N, e, c)
    print(libnum.n2s(m))

[1719620105458406433483340568317543019584575635895742560438771105058321655238562613083979651479555788009994557822024565226932906295208262756822275663694111, 52484065122572767557293534477361686456679280880304125291106733197354892893647364164212186415880889674435558369420400890814461263958618375991691022752189839]
b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 That is'
[920724637201, 159482692259010816139523195494724350795654007589889398757383554027183924116413427533184220914037106543253535103452324841452565420868944985464229649420240708554088156331324206733727690785373464575525698274552058386560106163093965065830071277465943834308083708065429495092746028681968670036721164931]
b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x07\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

### 低加密指数广播攻击
可以解出data3,data8,data12,data16,data20
解密结果：b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00t is a f'

In [6]:
from functools import reduce
def CRTAttack(Ns, e, cs):
    """
    低加密指数广播攻击（中国剩余定理）。
    原理：使用不同且互素的模数N、相同指数e且e较小，发送同一个数据m。根据中国剩余定理
    可以找到x满足x = c_i mod n_i forall i，对x开e次方就得到明文m。
    解出data3 data8 data12 data16 data20
    :param Ns:
    :param e:
    :param cs:
    :return: 明文m（数字形式） 或 None（攻击失败）
    """
    if libnum.gcd(*Ns) != 1:
        raise Exception("Input not pairwise co-prime")
    N_prod = reduce(lambda x, y: x * y, Ns)
    res = 0
    for c, N in zip(cs, Ns):
        m = N_prod // N
        m_inv = libnum.invmod(m, N)
        res += c * m * m_inv
        res %= N_prod
    m, succ = gmpy2.iroot(res, e)
    print(succ)
    return int(m)

Ns, es, cs = read_multiple_file([3, 8, 12, 16, 20])
m = CRTAttack(Ns, es[0], cs)
print(libnum.n2s(m))

True
b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00t is a f'


### Coppersmith已知部分填充攻击
可以解出data7,data11,data15
此为SageMath中可运行的脚本，鉴于在本地python程序中使用sage有点麻烦，建议访问https://sagecell.sagemath.org/在线运行
解密结果：
data7: b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00amous sa'
data11: b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00ying of '
data15: b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00Albert E'

In [10]:
b0 = libnum.s2n(b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')
b1 = libnum.s2n(b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')
delta = b1 - b0
print(f'b0: {b0}\n')
print(f'delta: {delta}\n')
Ns, es, cs = read_multiple_file([7, 11, 15])
for N, e, c in zip(Ns, es, cs):
    print(N, e, c, sep='\n')
    print('-'*20)

b0: 7985094500508197619216095178940144263456537191513943586246038968032293358555096757707390195545717847658989113846815132728257700850422861694332469740830720

delta: 169230328010303641331690318856389386196071598838855992136870091590247882556495704531248437872567112920983350278405979725889536

155266493936043103849855199987896813716831986416707080645036022909153373110367007140301635144950634879983289720164117794783088845393686109145443728632527874768524615377182297125716276153800765906014206797548230661764274997562670900115383324605843933035314110752560290540848152237316752573471110899212429555149
3
124929943232081828105808318993257526364596580021564021377503915670544445679836588765369503919311404328043203272693851622132258819278328852726005776082575583793735570095307898828254568015886630010269615546857335790791577865565021730890364239443651479580968112031521485174068731577348690810906553798608040451024
--------------------
11230606660165281906220643572479559560308590801100167118433222

In [9]:
def CoppersmithAttack(high_m, N, e, c):
    R.<x> = PolynomialRing(Zmod(N), implementation='NTL')
    m = high_m + x
    try:
        M = m((m^e - c).small_roots()[0])
        print(hex(int(M))[2:])
    except IndexError:
        print(m(m^e - c).small_roots())

N = 147733349387696521015664992396355145811249793103958464053225389476050097503928022819269482555955365534137156079172704297584033078453033637103720972881068435459202133846880715879894340131656691631756162323422868846616160423755883726450486845175227682329583615739797782025647376042249605775433971714513081755709
e = 3
c = 52253817590056116368273294519761274350847193477090280916373828903718796358618956145225746496960677477661151583828604021049936963779103440560630451125137344639503705880024677345063113240530798352727432768980751992926293801206779839157443722614687126711272753610923903360818026083573711899014859313677159790039
high_m = 7985094500508197619216095178940144263456537191513943586246038968032293358555096757707390195545717847658989113846815132728257700850422861694332469740830720
delta = 169230328010303641331690318856389386196071598838855992136870091590247882556495704531248437872567112920983350278405979725889536
for i in range(16):
    CoppersmithAttack(high_m, N, e, c)
    high_m += delta

SyntaxError: invalid syntax (<ipython-input-9-0902e1df9238>, line 3)

### 使用Fermat方法分解N
可以解出data10
解密结果：b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00will get'

In [24]:
def FermatFactor(n):
    """
    Fermat因数分解。将奇数n表示为n=a^2-b^2的形式，就能得到分解n=(a+b)(a-b)，通过枚举a的值寻找相应的b
    :param n:
    :return: ((a+b), (a-b))
    """
    def isqrt(n):
        """
        求整数的平方根
        :param n:
        :return:
        """
        x = n
        y = (x + n // x) // 2
        while y < x:
            x = y
            y = (x + n // x) // 2
        return x
    a = isqrt(n)
    b2 = a*a - n
    b = isqrt(n)
    while b*b != b2:
        a = a + 1
        b2 = a*a - n
        b = isqrt(b2)
    p=a+b
    q=a-b
    assert n == p * q
    return p, q

N, e, c = read_file("./dataset/data10")
p, q = FermatFactor(N)
r = (p - 1) * (q - 1)
d = libnum.invmod(e, r)
m = pow(c, d, N)
print(libnum.n2s(m))

b'\x98vT2\x10\xab\xcd\xef\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00will get'


## 以下为尝试后失败的方法，供参考
---
### 低加密指数爆破攻击
因为数据进行了填充，该方法不能解出任何一组数据

In [ ]:
import threading
def LowExpAttack(N, e, c, max_try, thread_num, start_at=0):
    """
    低加密指数爆破攻击。
    由c=m^e(mod N)得到m^e=k*N+c，枚举k再开e次方。最多枚举start_at+max_try*thread_num次
    :param N:
    :param e:
    :param c:
    :param max_try: 每一个线程，k的最多枚举次数
    :param thread_num: 线程数
    :param start_at: k的起始值
    :return: 明文m（数字形式） 或 None（攻击失败）
    """
    def calc(N, e, c, k_lower, k_upper, result, thread_idx):
        k = k_lower
        while k < k_upper:
            res = gmpy2.iroot(c + k * N, e)
            if res[1]:
                result.append(res[0])
                return
            k += 1
        print(f'Thread {thread_idx} which tests [{k_lower},{k_upper}) fails and ends')

    result = []
    # thread_num个线程, 线程i计算范围[start_at+max_try*i, start_at+max_try*(i+1)]
    threads = [threading.Thread(target=calc,
                                args=(
                                    N, e, c, start_at + max_try * i, start_at + max_try * (i + 1), result, i))
               for i in range(0, thread_num)]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    if len(result) == 0:
        print(f"All threads fail to k: {thread_num * max_try + start_at}")
        return None
    else:
        return result[0]

### Wiener攻击
要求$d<\frac{1}{3}N^\frac{1}{4}$
大概没有满足此要求的数据

In [ ]:
def WienerAttack(N, e):
    """
    Wiener攻击。利用连分数分解近似得到ed，再解二元一次方程得到d
    :param N:
    :param e:
    :param c:
    :return: (p,q,k,d)
    """
    def continued_fractions_expansion(e, N):
        """
        将e/N展开为连分数。
        :param e:
        :param N:
        :return:
        """
        result = []
        divident = e % N
        quotient = e // N
        result.append(quotient)
        while divident != 0:
            e -= quotient * N
            N, e = e, N
            divident = e % N
            quotient = e // N
            result.append(quotient)
        return result

    def convergents(expansion):
        convergents = [(expansion[0], 1)]
        for i in range(1, len(expansion)):
            numerator = 1
            denominator = expansion[i]
            for j in range(i - 1, -1, -1):
                numerator += expansion[j] * denominator
                if j == 0:
                    break
                tmp = denominator
                denominator = numerator
                numerator = tmp
            convergents.append((numerator, denominator))  # (k,d)
        return convergents

    cons = convergents(continued_fractions_expansion(e, N))
    for cs in cons:
        k, d = cs
        if k == 0:
            continue
        phi_N = (e * d - 1) // k
        # x**2 - ((N - phi_N) + 1) * x + N = 0
        a = 1
        b = -((N - phi_N) + 1)
        c = N
        delta = b * b - 4 * a * c
        if delta <= 0:
            continue
        root = int(gmpy2.iroot(delta, 2)[0])
        x1 = (root - b) // (2 * a)
        x2 = -(root + b) // (2 * a)
        if x1 * x2 == N:
            return [x1, x2, k, d]

### Pollard Rho, Pollard p-1, Williams p+1
都属于分解N的算法，参见primefac中的实现，测试中在短时间内没能得出剩余未解密数据中的任何一例

In [ ]:
primefac.pollardrho_brent()
primefac.pollard_pm1()
primefac.williams_pp1()